# Import and EDA

In [1]:
#importing libraries

#data
import pandas as pd
import numpy as np
import seaborn as sns
sns.set(style="ticks")

#time manipulation
from datetime import datetime, timedelta

#display setting
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.simplefilter("ignore")
pd.set_option('display.max_columns', 100)
from IPython.core.display import display, HTML

#plotly
import plotly
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.graph_objs as go
import plotly.figure_factory as ff
import plotly.subplots as sp

#MBA
!pip install mlxtend --upgrade --no-deps
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import fpgrowth
from mlxtend.frequent_patterns import association_rules

#graph
import networkx as nx
!pip install pyvis
from pyvis.network import Network

#KMeans
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler

#others
import os

%matplotlib inline

You should consider upgrading via the 'C:\ProgramData\Anaconda3\python.exe -m pip install --upgrade pip' command.
You should consider upgrading via the 'C:\ProgramData\Anaconda3\python.exe -m pip install --upgrade pip' command.


In [2]:
df = pd.read_csv('online_retail_data.csv', encoding= 'unicode_escape')

In [3]:
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate']).dt.date

# Helper functions

In [4]:
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

# Cleaning

In [5]:
mask = (~df['CustomerID'].isna())

In [6]:
df = df[mask]

In [7]:
df = df.drop_duplicates()

In [8]:
df = df.loc[df['StockCode'].str.match("\d{5}")]

In [9]:
customer_stockcode_date = df.groupby(['CustomerID','StockCode', 'InvoiceDate'])['Quantity'].sum().reset_index()

In [10]:
customer_stockcode_date = customer_stockcode_date[customer_stockcode_date['Quantity']>0]
customer_stockcode_date.head()

,CustomerID,StockCode,InvoiceDate,Quantity
1,12347.0,16008,2011-07-04,24
2,12347.0,17021,2011-09-06,36
3,12347.0,20665,2011-07-04,6
4,12347.0,20719,2011-01-26,10
5,12347.0,20719,2011-02-08,10


In [11]:
df = pd.merge(left=df,
              right=customer_stockcode_date[['CustomerID','StockCode','InvoiceDate']],
              how='inner',
              on = ['CustomerID','StockCode','InvoiceDate']
             )

In [12]:
#Product code dict
Products = df.drop_duplicates(subset='StockCode')[['StockCode','Description']]

Products = dict(zip(df['StockCode'], df['Description']))

# RFM Engineering

In [13]:
last_day = df['InvoiceDate'].max()

In [14]:
df['Sale'] = df['Quantity']*df['UnitPrice']

In [15]:
RFM_df = df.groupby('CustomerID').agg({'Sale':'sum', 'InvoiceNo':'nunique', 'InvoiceDate':'max', 'Country':'first'}).reset_index()

In [16]:
RFM_df = RFM_df.rename(columns={'Sale':'Monetary_Total', 'InvoiceNo':'Frequency', 'InvoiceDate':'LastPurchase'})

In [17]:
RFM_df['Recency'] = (RFM_df['LastPurchase'] - last_day).dt.days

In [18]:
RFM_df['Monetary_Average'] = RFM_df['Monetary_Total']/RFM_df['Frequency']

In [19]:
bin_labels = ['1', '2', '3', '4', '5']
RFM_df['Recency Score'] = pd.qcut(RFM_df['Recency'],
                              q=5,
                              labels=bin_labels)
RFM_df['Frequency Score'] = pd.qcut(RFM_df['Frequency'],
                             q=5,
                             labels=['2', '3', '4', '5'],
                             duplicates='drop')
RFM_df['Monetary Score'] = pd.qcut(RFM_df['Monetary_Average'],
                              q=5,
                              labels=bin_labels)

RFM_df['RFM_Score'] = RFM_df['Recency Score'].astype('str') + RFM_df['Frequency Score'].astype('str') + RFM_df['Monetary Score'].astype('str')

RFM_df = RFM_df.reindex(columns=['CustomerID','Monetary_Total','Recency','Frequency','Monetary_Average','LastPurchase',
                                 'Recency Score', 'Frequency Score','Monetary Score','RFM_Score', 'Country'
                                ])
RFM_df.head()

,CustomerID,Monetary_Total,Recency,Frequency,Monetary_Average,LastPurchase,Recency Score,Frequency Score,Monetary Score,RFM_Score,Country
0,12347.0,4310.00,-40,7,615.714286,2011-10-31,4,5,5,455,Iceland
1,12348.0,1437.24,-76,4,359.310000,2011-09-25,3,4,4,344,Finland
2,12349.0,1457.55,-19,1,1457.550000,2011-11-21,4,2,5,425,Italy
3,12350.0,294.40,-311,1,294.400000,2011-02-02,1,2,3,123,Norway
4,12352.0,1265.41,-73,6,210.901667,2011-09-28,3,4,2,342,Norway


In [20]:
# RFM Clustering

# # Segmentation

# RFM_df['Tier'] = 'Tier 4'
# RFM_df.loc[RFM_df['Monetary Score'].astype(int)>=3,'Tier'] = 'Tier 3'
# RFM_df.loc[RFM_df['Frequency Score'].astype(int)>=3,'Tier'] = 'Tier 2'
# RFM_df.loc[(RFM_df['Monetary Score'].astype(int)>=3)&(RFM_df['Frequency Score'].astype(int)>=4),'Tier'] = 'Tier 1'

# fig = px.scatter(RFM_df, x= 'Recency', y='Frequency', size='Monetary_Average', color='Tier', size_max=20)
# fig.update_layout({'plot_bgcolor': 'rgba(0, 0, 0, 0)',
#                    'paper_bgcolor': 'rgba(0, 0, 0, 0)'
#                   })
# fig.show()

# fig = px.scatter(RFM_df, x= 'Monetary_Average', y='Frequency', size='Monetary_Total', color='Tier', size_max=20)
# fig.update_layout({'plot_bgcolor': 'rgba(0, 0, 0, 0)',
#                    'paper_bgcolor': 'rgba(0, 0, 0, 0)'
#                   })
# fig.show()

# tiers = RFM_df.drop_duplicates(subset='Tier')[['CustomerID','Tier']]
# tiers = dict(zip(RFM_df['CustomerID'], RFM_df['Tier']))

# df['Tier'] = df['CustomerID'].map(tiers)

# K-Means clustering

In [21]:
# Perform K-means algorithm
norm = MinMaxScaler()
data = pd.DataFrame(norm.fit_transform(RFM_df[['Monetary_Average', 'Frequency']]), columns=['M_Scaled', 'F_Scaled'])

In [22]:
km_scores = []

for k in range(2, 11):
    km = KMeans(n_clusters=k, random_state=42)
    km.fit(data)
    km_scores.append(-km.score(data))

In [23]:
fig = px.line(x= range(2,11), y=km_scores)
fig.update_layout({'plot_bgcolor': 'rgba(0, 0, 0, 0)',
                   'paper_bgcolor': 'rgba(0, 0, 0, 0)'
                  })
fig.update_xaxes(title = 'Number of Clusters')
fig.update_yaxes(title = 'Clustering Score')
fig.show()

In [24]:
no_of_clusters = 5
model = KMeans(n_clusters=no_of_clusters, random_state=42)
cluster = pd.DataFrame(model.fit_predict(data)).astype('category')
cluster.columns = ['Cluster']
data = data.merge(cluster, left_index=True, right_index=True)
data = pd.concat([RFM_df,data], axis=1)
fig = px.scatter(data,
                 x='M_Scaled',
                 y='F_Scaled',
                 size = 'Monetary_Total',
                 color='Cluster')
fig.update_layout({'plot_bgcolor': 'rgba(0, 0, 0, 0)',
                   'paper_bgcolor': 'rgba(0, 0, 0, 0)'
                  })
fig.show()

In [25]:
Clusters = data.drop_duplicates(subset='Cluster')[['CustomerID','Cluster']]
Clusters = dict(zip(data['CustomerID'], data['Cluster']))

In [26]:
df['Cluster'] = df['CustomerID'].map(Clusters)

In [27]:
data.to_csv('Clustering_result.csv', index=False)

In [28]:
# percentage of non-UK customers
# data[data['Country']!='United Kingdom'].groupby('Cluster')['Country'].size()/data['Cluster'].value_counts()
data[data['Country']!='United Kingdom'].groupby('Cluster')['Country'].size()

Cluster
0    299
1     91
2     12
3      1
4     15
Name: Country, dtype: int64

# Market Basket

In [29]:
data['Cluster'].value_counts()

0    3737
1     340
4     226
2      23
3       7
Name: Cluster, dtype: int64

In [30]:
df['Cluster'].value_counts()

0    231428
4     83329
1     41148
3     24750
2     10468
Name: Cluster, dtype: int64

In [31]:
data = df.groupby(['Cluster', 'InvoiceNo'])['StockCode'].count().reset_index()
data = data[data['StockCode']>1]
# data.groupby('Cluster')['InvoiceNo'].count()
data.groupby('Cluster')['InvoiceNo'].nunique()

Cluster
0    10592
1     1137
2      210
3      837
4     4370
Name: InvoiceNo, dtype: int64

In [32]:
df.dtypes

InvoiceNo       object
StockCode       object
Description     object
Quantity         int64
InvoiceDate     object
UnitPrice      float64
CustomerID     float64
Country         object
Sale           float64
Cluster          int64
dtype: object

In [33]:
clusters = list(df['Cluster'].unique())

In [34]:
def MBA(df=df, min_support=0.04, min_lift=1.2, output_graph_name='MBA_graph.html', max_rule = None):
    print(f"***************************Cluster {cluster}***************************************")
    basket = (df.groupby(['InvoiceNo', 'StockCode'])['Quantity']
              .sum().unstack().reset_index().fillna(0)
              .set_index('InvoiceNo'))

    basket_sets = basket.applymap(encode_units)

    mask = basket_sets.sum(axis=1) > 1
    basket_sets = basket_sets[mask]

    frequent_itemsets = fpgrowth(
        basket_sets, min_support=min_support, use_colnames=True, max_len=2)
    rules = association_rules(
        frequent_itemsets, metric="lift", min_threshold=min_lift)
    # change product code to product name for readability
    rules['antecedents'] = rules['antecedents'].apply(lambda x: list(x)[0])
    rules['consequents'] = rules['consequents'].apply(lambda x: list(x)[0])
    rules['source'] = rules['antecedents'].map(Products)
    rules['target'] = rules['consequents'].map(Products)
    
    # write to csv
    filename = f'Cluster {cluster}.csv'
    rules.to_csv(filename)

    # prepare for graph visualisation
    rules_viz = rules.copy(deep=True)
    rules_viz['pair'] = rules_viz[['antecedents', 'consequents']].values.tolist()
    rules_viz['pair'] = rules_viz['pair'].apply(frozenset)
    rules_viz = rules_viz.drop_duplicates(subset='pair', keep='first')

    # for network graph
    rules_viz['value'] = rules_viz['lift']
    rules_viz['title'] = rules_viz['lift'].astype('str')
    if max_rule:
        rules_viz = rules_viz.sort_values(by = 'lift', ascending = False).head(max_rule)
    else:
        rules_viz = rules_viz.sort_values(by = 'lift', ascending = False)
    print(rules_viz.head())

    G = nx.from_pandas_edgelist(df=rules_viz,
                                source='source',
                                target='target',
                                edge_attr=['value','title']
                                )

    net = Network(height='700px', width='1000px', notebook=True)
    net.from_nx(G)
    net.show(output_graph_name)
    print("******************************************************************")

In [35]:
min_support_dict = {0: 0.03, 1:0.06, 2: 0.06, 3: 0.03, 4: 0.03}

for cluster in clusters:
    output_graph_name = f'result/Cluster {cluster}.html'
    MBA(df[df['Cluster'] == cluster], output_graph_name=output_graph_name,
        #         min_support=0.03,
        min_support=min_support_dict[cluster],
        min_lift=1,
        max_rule=100
        )

***************************Cluster 4***************************************
   antecedents consequents  antecedent support  consequent support   support  \
22       22699       22697            0.048070            0.037439  0.031431   
2        82482      82494L            0.052461            0.053848  0.030968   
0        22726       22727            0.062399            0.066790  0.042061   
34       23202       23203            0.063092            0.078345  0.039057   
8        22386       21931            0.071643            0.068408  0.031431   

    confidence       lift  leverage  conviction  \
22    0.653846  17.464150  0.029631    2.780731   
2     0.590308  10.962508  0.028143    2.309425   
0     0.674074  10.092452  0.037894    2.863258   
34    0.619048   7.901531  0.034114    2.419344   
8     0.438710   6.413165  0.026530    1.659733   

                               source                           target  \
22   ROSES REGENCY TEACUP AND SAUCER   GREEN REGENCY TEACUP AN

In [36]:
min_support_dict = {0: 0.02, 1:0.04, 2: 0.04, 3: 0.02, 4: 0.02}

for cluster in clusters:
    output_graph_name = f'result/Cluster {cluster}.html'
    MBA(df[df['Cluster'] == cluster], output_graph_name=output_graph_name,
        #         min_support=0.03,
        min_support=min_support_dict[cluster],
        min_lift=1,
        max_rule=100
        )

***************************Cluster 4***************************************
    antecedents consequents  antecedent support  consequent support   support  \
160       22698       22697            0.033511            0.037439  0.026346   
222       23300       23301            0.036515            0.042755  0.028426   
108       22699       22697            0.048070            0.037439  0.031431   
162       22699       22698            0.048070            0.033511  0.027271   
60        22910       22086            0.032355            0.040675  0.020106   

     confidence       lift  leverage  conviction  \
160    0.786207  20.999489  0.025092    4.502300   
222    0.778481  18.208040  0.026865    4.321278   
108    0.653846  17.464150  0.029631    2.780731   
162    0.567308  16.929244  0.025660    2.233665   
60     0.621429  15.277963  0.018790    2.534066   

                                 source                             target  \
160      PINK REGENCY TEACUP AND SAUCER    GRE